In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import vaex as vx

In [2]:
l = ['argentina','chile','china','panama','puerto_rico','norway','brazil','colombia','peru','australia','greece','UK','japan','south_africa','egypt','iceland','canada','mexico','portugal','spain','germany']
ld = {'argentina':'buenos_aires','chile':'santiago','china':'beijing','panama':'panama_city','puerto_rico':'san_juan','norway':'oslo','brazil':'brasilia','colombia':'bogota','peru':'lima','australia':'canberra','greece':'athens','UK':'london','japan':'tokyo','south_africa':'pretoria','egypt':'cairo','iceland':'reykjavik','canada':'ottawa','mexico':'mexico_city','portugal':'lisbon','spain':'madrid','germany':'berlin'}

In [3]:
df = vx.from_dict({'site_id':np.random.choice(l, size=1000), 'age':np.random.choice(range(18, 101), size=1000), 'weight':np.random.uniform(50, 95, size=1000),
                  'sex':np.random.choice(['F','M'], 1000), 'date':pd.date_range('2020-01-01', periods=1000, freq='60min')})

In [4]:
dc = pd.DataFrame({'site_id':l, 'city':list(ld.values())})

In [5]:
df = df.to_pandas_df()

In [6]:
#criando conexão db
con = sql.connect('teste.db')

In [7]:
#introduzindo datasets ao db
df.to_sql('users', con=con)
dc.to_sql('users2', con=con)

GROUP BY and ORDER BY

In [8]:
#média de idade por país
pd.read_sql_query("SELECT site_id, avg(age) AS age_avg \
                                 FROM users \
                                 GROUP BY site_id \
                                 ORDER BY avg(age) ", con)

,site_id,age_avg
0,panama,49.608696
1,china,51.886364
2,germany,54.940000
3,peru,55.232558
4,mexico,55.303571
5,iceland,55.531915
6,egypt,56.586957
7,UK,56.652174
8,colombia,57.136364
9,argentina,58.127660


In [9]:
#idade mínima de cada país
pd.read_sql_query("SELECT site_id, age AS age_min\
                                 FROM users \
                                 GROUP BY site_id \
                                 ORDER BY min(age) ", con)

,site_id,age_min
0,australia,18
1,brazil,18
2,canada,18
3,egypt,18
4,germany,18
5,iceland,18
6,japan,18
7,panama,18
8,spain,18
9,UK,19


In [10]:
#idade máxima de cada país
pd.read_sql_query("SELECT site_id, age AS age_max \
                     FROM users \
                     GROUP BY site_id \
                     ORDER BY max(age) ", con)

,site_id,age_max
0,argentina,93
1,china,93
2,chile,96
3,south_africa,97
4,australia,98
5,brazil,98
6,iceland,98
7,portugal,98
8,UK,99
9,canada,99


In [11]:
#contagem de linhas de cada país
pd.read_sql_query("SELECT site_id, count() AS site_count  \
                     FROM users \
                     GROUP BY site_id \
                     ORDER BY count() ", con)

,site_id,site_count
0,south_africa,37
1,greece,40
2,canada,43
3,peru,43
4,china,44
5,portugal,44
6,UK,46
7,brazil,46
8,chile,46
9,egypt,46


WHERE &nbsp;

between &nbsp;

in

In [12]:
#dados em que as idades estão entre 20 e 40 anos para o país brazil
pd.read_sql_query("SELECT site_id, age \
                     FROM users \
                     WHERE age between 20 and 40 and site_id == 'brazil' \
                     ORDER BY age ", con)

,site_id,age
0,brazil,22
1,brazil,23
2,brazil,28
3,brazil,28
4,brazil,29
5,brazil,32
6,brazil,36
7,brazil,37
8,brazil,40
9,brazil,40


In [13]:
#dados somentes dos países egypt e greece
pd.read_sql_query("SELECT site_id, age \
                     FROM users \
                     WHERE site_id in ('egypt', 'greece') \
                     ORDER BY age ", con)

,site_id,age
0,egypt,18
1,egypt,19
2,greece,21
3,greece,22
4,greece,22
...,...,...
81,greece,94
82,greece,98
83,egypt,98
84,greece,99


DISTINCT

In [14]:
#países em valores singulares
pd.read_sql_query("SELECT DISTINCT site_id \
                   FROM users", con)

,site_id
0,UK
1,peru
2,argentina
3,mexico
4,egypt
5,portugal
6,colombia
7,panama
8,japan
9,germany


WHERE with date value

In [15]:
#dados onde a data é maior que 2020-02-01
pd.read_sql_query("SELECT site_id, age, date \
                     FROM users \
                     WHERE date(date) > '2020-02-01' \
                     ORDER BY date ", con)

,site_id,age,date
0,canada,83,2020-02-02 00:00:00
1,iceland,57,2020-02-02 01:00:00
2,spain,97,2020-02-02 02:00:00
3,greece,21,2020-02-02 03:00:00
4,egypt,83,2020-02-02 04:00:00
...,...,...,...
227,portugal,88,2020-02-11 11:00:00
228,spain,47,2020-02-11 12:00:00
229,colombia,86,2020-02-11 13:00:00
230,mexico,49,2020-02-11 14:00:00


In [16]:
#média de idade para cada data do país
pd.read_sql_query("SELECT site_id, avg(age) AS age_avg, date(date) \
                     FROM users \
                     GROUP BY site_id,date(date)", con)

,site_id,age_avg,date(date)
0,UK,36.5,2020-01-01
1,UK,44.5,2020-01-02
2,UK,24.0,2020-01-04
3,UK,66.0,2020-01-06
4,UK,70.0,2020-01-07
...,...,...,...
618,spain,70.0,2020-02-06
619,spain,95.0,2020-02-08
620,spain,18.0,2020-02-09
621,spain,53.0,2020-02-10


CASE WHEN ELSE 

In [17]:
#nova coluna com novos valores baseados em condições de uma outra coluna
pd.read_sql_query("SELECT site_id, age, \
                   CASE WHEN site_id in ('brazil', 'peru') THEN 'X' ELSE 'S' END AS case_c \
                   FROM users ", con)

,site_id,age,case_c
0,UK,19,S
1,peru,26,X
2,argentina,50,S
3,mexico,89,S
4,egypt,26,S
...,...,...,...
995,portugal,88,S
996,spain,47,S
997,colombia,86,S
998,mexico,49,S


IFNULL and COALESCE

In [18]:
#substituindo valores null
pd.read_sql_query("SELECT site_id, age, \
                   IFNULL(CASE WHEN site_id in ('brazil', 'peru') THEN 'X' END, 'y') AS case_c \
                   FROM users ", con)

,site_id,age,case_c
0,UK,19,y
1,peru,26,X
2,argentina,50,y
3,mexico,89,y
4,egypt,26,y
...,...,...,...
995,portugal,88,y
996,spain,47,y
997,colombia,86,y
998,mexico,49,y


In [19]:
pd.read_sql_query("SELECT site_id, age, \
                   COALESCE(CASE WHEN site_id in ('brazil', 'peru') THEN 'X' END, 'y ') AS case_c \
                   FROM users ", con)

,site_id,age,case_c
0,UK,19,y
1,peru,26,X
2,argentina,50,y
3,mexico,89,y
4,egypt,26,y
...,...,...,...
995,portugal,88,y
996,spain,47,y
997,colombia,86,y
998,mexico,49,y


INNER JOIN

In [20]:
#combinando valores de outra tabela
pd.read_sql_query("SELECT users.site_id, city, age \
                       FROM users \
                       INNER JOIN users2 ON users.site_id = users2.site_id", con)

,site_id,city,age
0,UK,london,19
1,peru,lima,26
2,argentina,buenos_aires,50
3,mexico,mexico_city,89
4,egypt,cairo,26
...,...,...,...
995,portugal,lisbon,88
996,spain,madrid,47
997,colombia,bogota,86
998,mexico,mexico_city,49


In [21]:
pd.read_sql_query("SELECT users2.site_id, city, age \
                   FROM users2 \
                   INNER JOIN users ON users2.site_id = users.site_id", con)

,site_id,city,age
0,argentina,buenos_aires,22
1,argentina,buenos_aires,23
2,argentina,buenos_aires,25
3,argentina,buenos_aires,27
4,argentina,buenos_aires,28
...,...,...,...
995,germany,berlin,92
996,germany,berlin,94
997,germany,berlin,94
998,germany,berlin,95


CONCATENATE

In [22]:
#concatenando valores de colunas diferentes
pd.read_sql_query("SELECT users.site_id||' <•> '||city AS country_city, age \
                   FROM users \
                   INNER JOIN users2 ON users2.site_id = users.site_id", con)

,country_city,age
0,UK <•> london,19
1,peru <•> lima,26
2,argentina <•> buenos_aires,50
3,mexico <•> mexico_city,89
4,egypt <•> cairo,26
...,...,...
995,portugal <•> lisbon,88
996,spain <•> madrid,47
997,colombia <•> bogota,86
998,mexico <•> mexico_city,49


UPDATE and DELETE

In [23]:
#alterando valor de um dado específico
con.execute("UPDATE users SET site_id == 'united_kingdom' WHERE site_id == 'UK' ")
pd.read_sql_query("SELECT site_id, age \
                   FROM users \
                   WHERE site_id == 'united_kingdom'", con)

,site_id,age
0,united_kingdom,19
1,united_kingdom,54
2,united_kingdom,27
3,united_kingdom,62
4,united_kingdom,24
5,united_kingdom,66
6,united_kingdom,86
7,united_kingdom,59
8,united_kingdom,65
9,united_kingdom,39


In [24]:
#deletando dados de um país
con.execute("DELETE FROM users WHERE site_id == 'norway' ")
pd.read_sql_query("SELECT site_id \
                   FROM users \
                   GROUP BY site_id", con)

,site_id
0,argentina
1,australia
2,brazil
3,canada
4,chile
5,china
6,colombia
7,egypt
8,germany
9,greece
